In [4]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
#from utee import misc, quant, selector

import torch.nn.functional as F  # useful stateless functions

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np


#Load CIFAR-10
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [6]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

#test_flatten()

In [7]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [8]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    t_begin = time.time()
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            

            if t % print_every == 0:
                t_elapse = time.time() - t_begin
                print('Elapsed %.4f s, Epoch %d,  Iteration %d, loss = %.4f' % (t_elapse, e, t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [9]:
################################################################################
# TODO:                                                                        #         
# Experiment with any architectures, optimizers, and hyperparameters.          #
# Achieve AT LEAST 70% accuracy on the *validation set* within 10 epochs.      #
#                                                                              #
# Note that you can use the check_accuracy function to evaluate on either      #
# the test set or the validation set, by passing either loader_test or         #
# loader_val as the second argument to check_accuracy. You should not touch    #
# the test set until you have finished your architecture and  hyperparameter   #
# tuning, and only run the test set once at the end to report a final value.   #
################################################################################

'''
model = nn.Sequential(
    nn.Conv2d(3, 32, 5,stride=1, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2,stride=2),
    nn.Conv2d(32, 64, 3,stride=1,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2,stride=2),
    Flatten(),
    nn.Linear(4096,512),
    nn.ReLU(),
    nn.Linear(512,10),
)
'''

class ExpConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a three-layer ConvNet with the  #
        # architecture defined above.                                          #
        ########################################################################
        self.conv1 = nn.Conv2d(3, 32, 5, stride=1, padding=2)
        nn.init.kaiming_normal_(self.conv1.weight)
        nn.init.constant_(self.conv1.bias, 0)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        nn.init.kaiming_normal_(self.conv2.weight)
        nn.init.constant_(self.conv2.bias, 0)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64*8*8, 512)
        self.fc2 = nn.Linear(512, 10)
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.constant_(self.fc2.bias, 0)
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        x = self.conv1(x)
        #x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(F.relu(x))
        x = self.conv2(x)
        #x = self.bn2(x)
        x = F.relu(x)
        x = self.maxpool(F.relu(x))
        x = flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        
        scores = x
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores


In [17]:
model = ExpConvNet()
optimizer = optim.Adam(params=model.parameters(), lr=1e-3)

################################################################################
#                                 END OF YOUR CODE                             
################################################################################

# You should get at least 70% accuracy
train_part34(model, optimizer, epochs=5)

Elapsed 0.0134 s, Epoch 0,  Iteration 0, loss = 5.7583
Checking accuracy on validation set
Got 113 / 1000 correct (11.30)

Elapsed 1.3902 s, Epoch 0,  Iteration 100, loss = 1.8266
Checking accuracy on validation set
Got 383 / 1000 correct (38.30)

Elapsed 2.7424 s, Epoch 0,  Iteration 200, loss = 1.4599
Checking accuracy on validation set
Got 482 / 1000 correct (48.20)

Elapsed 4.1081 s, Epoch 0,  Iteration 300, loss = 1.3743
Checking accuracy on validation set
Got 503 / 1000 correct (50.30)

Elapsed 5.4676 s, Epoch 0,  Iteration 400, loss = 1.1487
Checking accuracy on validation set
Got 525 / 1000 correct (52.50)

Elapsed 6.8207 s, Epoch 0,  Iteration 500, loss = 1.3694
Checking accuracy on validation set
Got 558 / 1000 correct (55.80)

Elapsed 8.1550 s, Epoch 0,  Iteration 600, loss = 1.2151
Checking accuracy on validation set
Got 600 / 1000 correct (60.00)

Elapsed 9.4865 s, Epoch 0,  Iteration 700, loss = 1.1405
Checking accuracy on validation set
Got 587 / 1000 correct (58.70)

El

In [18]:
torch.save(model.state_dict(), '../pretrain_model/training.pt')

In [19]:
from Model.quantization import *

class FixedLayerConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a three-layer ConvNet with the  #
        # architecture defined above.                                          #
        ########################################################################
        self.bits = 8
        self.conv1 = nn.Conv2d(3, 32, 5, stride=1, padding=2)
        #nn.init.kaiming_normal_(self.conv1.weight)
        #nn.init.constant_(self.conv1.bias, 0)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        #nn.init.kaiming_normal_(self.conv2.weight)
        #nn.init.constant_(self.conv2.bias, 0)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        #self.bn2 = nn.BatchNorm2d(512)
        self.fc1 = nn.Linear(64*8*8, 512)
        self.fc2 = nn.Linear(512, 10)
        #nn.init.kaiming_normal_(self.fc.weight)
        self.quant = activation_quantization(8, Quant.linear)
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        #x = quantization.apply(x, self.bits, Quant.linear)
        x = self.quant(x)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.maxpool(F.relu(x))
        #x = quantization.apply(x, self.bits, Quant.linear)
        x = self.quant(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.maxpool(F.relu(x))
        x = flatten(x)
        x = self.quant(x)
        #x = quantization.apply(x, self.bits, Quant.linear)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.quant(x)
        #x = quantization.apply(x, self.bits, Quant.linear)
        x = self.fc2(x)
        
        scores = x
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores

fix_model = FixedLayerConvNet()
direct_fix_model = FixedLayerConvNet()

In [20]:
PATH = '../pretrain_model/training.pt'
model_pretrain = torch.load(PATH)
fix_model
#model_pretrain
#for k, v in model.module.state_dict().items():

FixedLayerConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (quant): activation_quantization()
)

In [21]:
#Load module to loadmodel
PATH = '../pretrain_model/training.pt'
fix_model.load_state_dict(torch.load(PATH))
direct_fix_model.load_state_dict(torch.load(PATH))

#Solve the weight type problem, change to cudafloat tensor
if USE_GPU:
    fix_model.cuda()
    fix_model = torch.nn.DataParallel(fix_model, device_ids=range(torch.cuda.device_count()))

if USE_GPU:
    direct_fix_model.cuda()
    direct_fix_model = torch.nn.DataParallel(direct_fix_model, device_ids=range(torch.cuda.device_count()))
    #cudnn.benchmark = True

In [22]:
#Train this model
learning_rate = 2e-6

optimizer = optim.Adam(params=fix_model.parameters(), lr=learning_rate)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9, nesterov=True)
train_part34(fix_model, optimizer, epochs=10)


Elapsed 0.0177 s, Epoch 0,  Iteration 0, loss = 0.3403
Checking accuracy on validation set
Got 659 / 1000 correct (65.90)

Elapsed 1.6932 s, Epoch 0,  Iteration 100, loss = 0.2090
Checking accuracy on validation set
Got 663 / 1000 correct (66.30)

Elapsed 3.3739 s, Epoch 0,  Iteration 200, loss = 0.3896
Checking accuracy on validation set
Got 667 / 1000 correct (66.70)

Elapsed 5.0427 s, Epoch 0,  Iteration 300, loss = 0.3283
Checking accuracy on validation set
Got 671 / 1000 correct (67.10)

Elapsed 6.7102 s, Epoch 0,  Iteration 400, loss = 0.2870
Checking accuracy on validation set
Got 671 / 1000 correct (67.10)

Elapsed 8.3894 s, Epoch 0,  Iteration 500, loss = 0.3506
Checking accuracy on validation set
Got 674 / 1000 correct (67.40)

Elapsed 10.0648 s, Epoch 0,  Iteration 600, loss = 0.2843
Checking accuracy on validation set
Got 672 / 1000 correct (67.20)

Elapsed 11.7585 s, Epoch 0,  Iteration 700, loss = 0.2298
Checking accuracy on validation set
Got 676 / 1000 correct (67.60)



Got 689 / 1000 correct (68.90)

Elapsed 107.3777 s, Epoch 8,  Iteration 200, loss = 0.2693
Checking accuracy on validation set
Got 687 / 1000 correct (68.70)

Elapsed 109.0759 s, Epoch 8,  Iteration 300, loss = 0.3329
Checking accuracy on validation set
Got 690 / 1000 correct (69.00)

Elapsed 110.7750 s, Epoch 8,  Iteration 400, loss = 0.2182
Checking accuracy on validation set
Got 681 / 1000 correct (68.10)

Elapsed 112.4724 s, Epoch 8,  Iteration 500, loss = 0.1640
Checking accuracy on validation set
Got 691 / 1000 correct (69.10)

Elapsed 114.1627 s, Epoch 8,  Iteration 600, loss = 0.3199
Checking accuracy on validation set
Got 679 / 1000 correct (67.90)

Elapsed 115.8596 s, Epoch 8,  Iteration 700, loss = 0.2301
Checking accuracy on validation set
Got 683 / 1000 correct (68.30)

Elapsed 117.0332 s, Epoch 9,  Iteration 0, loss = 0.2050
Checking accuracy on validation set
Got 681 / 1000 correct (68.10)

Elapsed 118.7322 s, Epoch 9,  Iteration 100, loss = 0.2296
Checking accuracy on v

In [23]:
print("Finetune Fixed Point Accuracy:")
check_accuracy_part34(loader_test, fix_model)
print("\nDirect Fixed Point Accuracy:")
check_accuracy_part34(loader_test, direct_fix_model)
print("\nOriginal Floating Point Accuracy:")
check_accuracy_part34(loader_test, model)

Finetune Fixed Point Accuracy:
Checking accuracy on test set
Got 6876 / 10000 correct (68.76)

Direct Fixed Point Accuracy:
Checking accuracy on test set
Got 6812 / 10000 correct (68.12)

Original Floating Point Accuracy:
Checking accuracy on test set
Got 6950 / 10000 correct (69.50)


In [ ]:
#next step: 
#1. implement the weight fixed 
#2. use the code provide by playground in the forward function
#3. implement the convfixed layer, and fc fixed layer, bn fixed layer

In [24]:
class FixedConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a three-layer ConvNet with the  #
        # architecture defined above.                                          #
        ########################################################################
        self.bits = 8
        self.conv1 = nn.Conv2d(3, 32, 5, stride=1, padding=2)
        #nn.init.kaiming_normal_(self.conv1.weight)
        #nn.init.constant_(self.conv1.bias, 0)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        #nn.init.kaiming_normal_(self.conv2.weight)
        #nn.init.constant_(self.conv2.bias, 0)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        #self.bn2 = nn.BatchNorm2d(512)
        self.fc1 = nn.Linear(64*8*8, 512)
        self.fc2 = nn.Linear(512, 10)
        #nn.init.kaiming_normal_(self.fc.weight)
        self.act_quant = activation_quantization(8, Quant.linear)
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        x = self.act_quant(x)
        
        #self.conv1.weight = torch.nn.Parameter(linear_quantize(self.conv1.weight, 8))
        #self.conv1.bias = torch.nn.Parameter(linear_quantize(self.conv1.bias, 8))
        x = self.conv1(x)
        #x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(F.relu(x))
        x = self.act_quant(x)
        #self.conv2.weight = torch.nn.Parameter(linear_quantize(self.conv2.weight, 8))
        #self.conv2.bias = torch.nn.Parameter(linear_quantize(self.conv2.bias, 8))
        x = self.conv2(x)
        #x = self.bn2(x)
        x = F.relu(x)
        x = self.maxpool(F.relu(x))
        x = flatten(x)
        
        x = self.act_quant(x)
        #self.fc1.weight = torch.nn.Parameter(linear_quantize(self.fc1.weight, 8))
        x = self.fc1(x)
        x = F.relu(x)
        x = self.act_quant(x)
        #self.fc2.weight = torch.nn.Parameter(linear_quantize(self.fc2.weight, 8))
        x = self.fc2(x)
        
        scores = x
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores


In [28]:
#TODO: 1. Implement different function
#      2. differentiate FC and CONV
def quantize_weight(model, bits):
    for k, v in model.module.state_dict().items():
        model.module.state_dict()[k] = Quant.linear(v, bits)
    return model

#TODO: Add a dictionary for bit width and function.
def train_fixed_weight(model, optimizer, epochs=1, bits=8):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    t_begin = time.time()
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            
            #Add quantization for weight
            model = quantize_weight(model, bits)

            if t % print_every == 0:
                t_elapse = time.time() - t_begin
                print('Elapsed %.4f s, Epoch %d,  Iteration %d, loss = %.4f' % (t_elapse, e, t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [29]:
#Load module to loadmodel
PATH = '../pretrain_model/training.pt'

model = ExpConvNet()
fix_model = FixedConvNet()
direct_fix_model = FixedConvNet()

model.load_state_dict(torch.load(PATH))
fix_model.load_state_dict(torch.load(PATH))
direct_fix_model.load_state_dict(torch.load(PATH))

#Solve the weight type problem, change to cudafloat tensor
if USE_GPU:
    fix_model.cuda()
    fix_model = torch.nn.DataParallel(fix_model, device_ids=range(torch.cuda.device_count()))

if USE_GPU:
    direct_fix_model.cuda()
    direct_fix_model = torch.nn.DataParallel(direct_fix_model, device_ids=range(torch.cuda.device_count()))
    #cudnn.benchmark = True

if USE_GPU:
    model.cuda()
    model = torch.nn.DataParallel(direct_fix_model, device_ids=range(torch.cuda.device_count()))

quantize_weight(fix_model, 8)
quantize_weight(direct_fix_model, 8)

DataParallel(
  (module): FixedConvNet(
    (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=4096, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=10, bias=True)
    (act_quant): activation_quantization()
  )
)

In [34]:
#Train this model
learning_rate = 5e-7

optimizer = optim.Adam(params=fix_model.parameters(), lr=learning_rate)
#optimizer = optim.SGD(fix_model.parameters(), lr=learning_rate,momentum=0.9, nesterov=True)
train_fixed_weight(fix_model, optimizer, epochs=5)

Elapsed 0.0297 s, Epoch 0,  Iteration 0, loss = 0.2896
Checking accuracy on validation set
Got 672 / 1000 correct (67.20)

Elapsed 2.9527 s, Epoch 0,  Iteration 100, loss = 0.3342
Checking accuracy on validation set
Got 675 / 1000 correct (67.50)

Elapsed 5.8740 s, Epoch 0,  Iteration 200, loss = 0.2804
Checking accuracy on validation set
Got 678 / 1000 correct (67.80)

Elapsed 8.7894 s, Epoch 0,  Iteration 300, loss = 0.3075
Checking accuracy on validation set
Got 676 / 1000 correct (67.60)

Elapsed 11.7196 s, Epoch 0,  Iteration 400, loss = 0.2005
Checking accuracy on validation set
Got 677 / 1000 correct (67.70)

Elapsed 14.6372 s, Epoch 0,  Iteration 500, loss = 0.2506
Checking accuracy on validation set
Got 675 / 1000 correct (67.50)

Elapsed 17.5648 s, Epoch 0,  Iteration 600, loss = 0.2783
Checking accuracy on validation set
Got 678 / 1000 correct (67.80)

Elapsed 20.6182 s, Epoch 0,  Iteration 700, loss = 0.1880
Checking accuracy on validation set
Got 677 / 1000 correct (67.70)

In [33]:
print("Finetune Fixed Point Accuracy:")
check_accuracy_part34(loader_test, fix_model)
print("\nDirect Fixed Point Accuracy:")
check_accuracy_part34(loader_test, direct_fix_model)
print("\nOriginal Floating Point Accuracy:")
check_accuracy_part34(loader_test, model)

Finetune Fixed Point Accuracy:
Checking accuracy on test set
Got 6876 / 10000 correct (68.76)

Direct Fixed Point Accuracy:
Checking accuracy on test set
Got 6812 / 10000 correct (68.12)

Original Floating Point Accuracy:
Checking accuracy on test set
Got 6812 / 10000 correct (68.12)


In [220]:
print(direct_fix_model.module.state_dict()['conv1.weight']*1024)

tensor([[[[-198.2814,  255.2448,  117.6480, -207.7786, -189.1101],
          [-103.1326,  251.6528,  -79.1741, -319.9017,  -12.8926],
          [ 235.1852,   71.4552,  -19.8977, -286.5714,  216.7204],
          [ 133.1982,  -64.3097, -264.7648,  126.2105,  -41.5371],
          [  98.8290, -175.6736,  142.7939,  146.3576,  -59.8986]],

         [[-251.8881, -104.8649, -257.0427,   79.2322,  175.4056],
          [ -75.6155,   30.4536,  -73.4142,   42.4973,  243.2123],
          [ 227.3108, -119.0568,   55.2801,   95.9392,  -11.9220],
          [ 261.6974,   89.3977, -313.3453,  363.6068,  150.8462],
          [ 256.3130, -276.7975, -355.8281,  401.6050,   18.7703]],

         [[  46.9333,  237.2028,  129.9361,   20.4777,   22.2818],
          [-229.5939,  169.2838,   20.6609,  111.7582,    4.7855],
          [-165.5342,  -91.6710,   -9.8183,  -51.4141,  -62.1809],
          [ 337.9961,  -96.6034, -162.7282,  -78.8396,   31.5714],
          [  37.1734,   -9.6663, -127.2759, -253.4323, -14